In [1]:
#импорт библиотек
import cv2
import numpy as np
import math

#размеры окна
WIND_X = 600
WIND_Y = 500

g = 0.3

#базовый класс
class Figure:
    def __init__ (self, x_, y_, vx_, vy_):
        self.x  = x_
        self.y  = y_
        self.vx = vx_
        self.vy = vy_

    def draw (self, canvas):
        pass
    
    def move (self):
        self.x += self.vx
        self.y += self.vy
        
        if (self.x < 0 or self.x > WIND_X):
            self.vx *= -1

        if (self.y < 0 or self.y > WIND_Y):
            self.vy *= -1

#производный класс Круг
class Circle (Figure):
    def __init__ (self, x_, y_, vx_, vy_, r_, color_by_coords_, ay_ = g):
        Figure.__init__ (self, x_, y_, vx_, vy_)
        
        self.r = r_
        self.ay = ay_
        self.color_by_coords = color_by_coords_
        
    def draw (self, canvas):
        color = self.color_by_coords (self.x, self.y, self.vx, self.vy)
        
        cv2.circle (canvas, (int (self.x), int (self.y)), int (self.r), color, 1)
    
    def move (self):
        if (self.x < self.r or self.x > WIND_X - self.r):
            self.vx *= -1

        if (self.y < self.r or self.y > WIND_Y - self.r):
            self.vy *= -1
            self.vy -= self.ay

        self.vy += self.ay

        self.x += int (self.vx)
        self.y += int (self.vy)
                
#производный класс Квадрат
class Square (Figure):
    def __init__ (self, x_, y_, vx_, vy_, sz_):
        Figure.__init__ (self, x_, y_, vx_, vy_)
        
        self.sz = sz_
        
    def draw (self, canvas):
        cv2.rectangle (canvas, (self.x, self.y), (self.x + self.sz, self.y + self.sz),
            (10, 120, 50), -1)
    
    def move (self):
        self.x += self.vx
        self.y += self.vy
        
        if (self.x < 0 or self.x > WIND_X - self.sz):
            self.vx *= -1

        if (self.y < 0 or self.y > WIND_Y - self.sz):
            self.vy *= -1

#производный класс Линия
class Line (Figure):
    def __init__ (self, x1_, y1_, vx1_, vy1_, x2_, y2_, vx2_, vy2_):
        Figure.__init__ (self, x1_, y1_, vx1_, vy1_)
        
        self.fig1 = Figure (x1_, y1_, vx1_, vy1_)
        self.fig2 = Figure (x2_, y2_, vx2_, vy2_)
        
    def draw (self, canvas):
        cv2.line (canvas, (self.fig1.x, self.fig1.y),
            (self.fig2.x, self.fig2.y),
            ((255 + int (self.fig1.x * 0.3)) % 255,
             (210 + int (self.fig2.y * 0.56)) % 255, 130 +
             np.random.randint (5, 20)), 2)
    
    def move (self):
        self.fig1.move ()
        self.fig2.move ()

#менеджер
class Manager:
    def __init__ (self, canvas_ref_):
        #лист со всеми фигурами
        self.figures = []
        
        self.refresh_speed = 5
        self.pen_coord = 10
        self.graph_canvas = canvas_ref_.copy ()
        
        self.Ek = 0
        self.Ep = 0
    
    def move (self):
        #движение
        for fig in self.figures:
            fig.move ()

    def draw (self, canvas):
        #рисование
        for fig in self.figures:
            fig.draw (canvas)

    def add_figure (self, figure):
        self.figures.append (figure)
    
    def handle_collisions (self):
        for i in range (len (self.figures)):
            for j in range (i + 1, len (self.figures)):
                fig1 = self.figures [i]
                fig2 = self.figures [j]
                
                distance = math.sqrt ((fig1.x - fig2.x)**2 + (fig1.y - fig2.y)**2)
                
                if (distance <= fig1.r + fig2.r):
                    vx1     = fig1.vx
                    fig1.vx = fig2.vx
                    fig2.vx = vx1
                    
                    vy1     = fig1.vy
                    fig1.vy = fig2.vy
                    fig2.vy = vy1
    
    def visualize_energy (self, scale = 3):
        self.pen_coord += self.refresh_speed
        
        if (self.pen_coord >= WIND_X - 10):
            self.pen_coord = 10
        
        cv2.circle (self.graph_canvas, (self.pen_coord, int (WIND_Y - self.Ek * scale)),
                    3, (100, 200, 0), -1)

        cv2.circle (self.graph_canvas, (self.pen_coord, int (WIND_Y - self.Ep * scale)),
                    3, (200, 20, 0), -1)

        #cv2.circle (self.graph_canvas, (self.pen_coord,
        #    int (WIND_Y - (self.Ek + self.Ep) * scale)), 3, (255, 255, 255), -1)
        
        return self.graph_canvas
    
    def calc_energy (self):
        self.Ek = 0
        self.Ep = 0
        
        for fig in self.figures:
            self.Ek += (float (fig.vx)**2 + fig.vy**2) / 2
            self.Ep += g * (float (WIND_Y) - fig.r - fig.y)
        
        print ("Ek, Ep: ", self.Ek, self.Ep)

def color_by_coord_CHAOTIC_EVIL (x, y, vx, vy):
    return ((x * 3) % 255,
            (y * 4 + vx * 10) % 255,
            (vy * 20 + 100) % 255)

def color_by_coord_shades_of_gray (x, y, vx, vy):
    return ((y + vx * 6) % 255,
            (y + vx * 5) % 255,
            (y + vx * 4) % 255)

#холст
canvas = np.ones ((WIND_Y, WIND_X, 3), np.uint8) * 60

manager = Manager (canvas)

#добавление новой
#figures.append (Circle (100, 200, 3, 5, 50))
#figures.append (Square (300, 200, -1, 3, 60))
#figures.append (Line   (300, 200, -1, 3, 400, 100, 2, 5))

#цикл (основная часть программы)
while (True):
    #движение
    manager.move ()
    manager.handle_collisions ()
    manager.calc_energy ()
    
    #визуализация
    manager.draw (canvas)

    #вывод
    graphs = manager.visualize_energy ()
    frame = np.concatenate ((canvas, graphs), axis=1)
    
    cv2.imshow ("frame", frame)
    
    #обработка клавиатуры
    key = cv2.waitKey (1) & 0xFF
    
    if (key == ord ('q')):
        break
    
    vx1 = float (np.random.randint (2, 4))
    vy1 = float (np.random.randint (1, 5))
        
    if (key == ord ('a')):
        manager.add_figure (Circle (300.0, 200.0, vx1, vy1,
            40.0, color_by_coord_CHAOTIC_EVIL))

    if (key == ord ('s')):
        manager.add_figure (Circle (300.0, 200.0, vx1, vy1,
            30.0, color_by_coord_shades_of_gray))


#закрытие окон
cv2.destroyAllWindows ()

Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0
Ek, Ep:  0 0

Ek, Ep:  7.779999999999995 80.7
Ek, Ep:  8.844999999999995 79.8
Ek, Ep:  9.999999999999993 78.89999999999999
Ek, Ep:  11.244999999999992 77.7
Ek, Ep:  12.57999999999999 76.5
Ek, Ep:  14.004999999999988 75.3
Ek, Ep:  15.519999999999987 73.8
Ek, Ep:  17.124999999999986 72.3
Ek, Ep:  18.819999999999983 70.8
Ek, Ep:  20.604999999999983 69.0
Ek, Ep:  22.47999999999998 67.2
Ek, Ep:  24.44499999999998 65.39999999999999
Ek, Ep:  26.499999999999975 63.599999999999994
Ek, Ep:  28.64499999999997 61.5
Ek, Ep:  30.87999999999997 59.4
Ek, Ep:  33.20499999999997 57.3
Ek, Ep:  35.61999999999996 54.9
Ek, Ep:  38.12499999999997 52.5
Ek, Ep:  40.71999999999998 50.1
Ek, Ep:  43.40499999999998 47.4
Ek, Ep:  46.179999999999986 44.699999999999996
Ek, Ep:  49.044999999999995 42.0
Ek, Ep:  52.0 39.0
Ek, Ep:  55.04500000000001 36.0
Ek, Ep:  58.180000000000014 33.0
Ek, Ep:  61.40500000000002 30.0
Ek, Ep:  64.72000000000003 26.7
Ek, Ep:  68.12500000000004 23.4
Ek, Ep:  71.62000000000005 20.099999999999998
Ek, Ep:

Ek, Ep:  12.57999999999999 77.7
Ek, Ep:  11.244999999999992 78.89999999999999
Ek, Ep:  9.999999999999993 79.8
Ek, Ep:  8.844999999999995 80.7
Ek, Ep:  7.779999999999995 81.6
Ek, Ep:  6.804999999999996 82.5
Ek, Ep:  5.919999999999996 83.1
Ek, Ep:  5.124999999999998 83.7
Ek, Ep:  4.419999999999998 84.3
Ek, Ep:  3.804999999999999 84.6
Ek, Ep:  3.2799999999999985 84.89999999999999
Ek, Ep:  2.844999999999999 85.2
Ek, Ep:  2.499999999999999 85.2
Ek, Ep:  2.244999999999999 85.2
Ek, Ep:  2.0799999999999996 85.2
Ek, Ep:  2.005 85.2
Ek, Ep:  2.02 85.2
Ek, Ep:  2.1250000000000004 85.2
Ek, Ep:  2.3200000000000007 85.2
Ek, Ep:  2.6050000000000013 84.89999999999999
Ek, Ep:  2.9800000000000013 84.6
Ek, Ep:  3.445000000000002 84.3
Ek, Ep:  4.000000000000002 83.7
Ek, Ep:  4.645000000000001 83.1
Ek, Ep:  5.380000000000001 82.5
Ek, Ep:  6.205000000000001 81.89999999999999
Ek, Ep:  7.120000000000001 81.0
Ek, Ep:  8.125 80.1
Ek, Ep:  9.219999999999999 79.2
Ek, Ep:  10.405 78.0
Ek, Ep:  11.679999999999998 7

Ek, Ep:  72.15000000000003 110.1
Ek, Ep:  70.26000000000002 111.9
Ek, Ep:  68.55000000000001 113.7
Ek, Ep:  67.02000000000001 115.19999999999999
Ek, Ep:  65.66999999999999 116.4
Ek, Ep:  64.49999999999999 117.6
Ek, Ep:  63.50999999999998 118.5
Ek, Ep:  62.699999999999974 119.1
Ek, Ep:  62.069999999999965 119.7
Ek, Ep:  61.61999999999996 120.0
Ek, Ep:  61.349999999999966 120.0
Ek, Ep:  61.25999999999996 120.0
Ek, Ep:  61.349999999999966 120.0
Ek, Ep:  61.61999999999996 119.69999999999999
Ek, Ep:  62.069999999999965 119.1
Ek, Ep:  62.699999999999974 118.5
Ek, Ep:  63.50999999999998 117.6
Ek, Ep:  64.49999999999999 116.39999999999999
Ek, Ep:  65.66999999999999 115.19999999999999
Ek, Ep:  67.02000000000001 113.69999999999999
Ek, Ep:  68.55000000000001 111.89999999999999
Ek, Ep:  70.26000000000002 110.1
Ek, Ep:  72.15000000000003 108.3
Ek, Ep:  74.22000000000004 105.89999999999999
Ek, Ep:  76.47000000000006 103.5
Ek, Ep:  78.90000000000006 101.1
Ek, Ep:  81.51000000000006 98.4
Ek, Ep:  84.3

Ek, Ep:  51.524999999999984 126.9
Ek, Ep:  54.824999999999996 123.89999999999999
Ek, Ep:  58.305 120.6
Ek, Ep:  61.965 117.0
Ek, Ep:  65.805 113.39999999999999
Ek, Ep:  69.82500000000002 109.8
Ek, Ep:  74.02500000000002 105.6
Ek, Ep:  78.40500000000003 101.4
Ek, Ep:  82.96500000000005 97.19999999999999
Ek, Ep:  87.70500000000004 92.69999999999999
Ek, Ep:  92.62500000000006 87.9
Ek, Ep:  97.72500000000007 83.1
Ek, Ep:  103.00500000000008 78.0
Ek, Ep:  108.46500000000009 72.6
Ek, Ep:  114.1050000000001 67.2
Ek, Ep:  119.92500000000011 61.8
Ek, Ep:  125.92500000000013 55.8
Ek, Ep:  132.10500000000013 49.8
Ek, Ep:  138.46500000000015 43.8
Ek, Ep:  140.52000000000015 46.199999999999996
Ek, Ep:  138.27000000000012 48.3
Ek, Ep:  136.2000000000001 50.39999999999999
Ek, Ep:  134.31000000000012 52.2
Ek, Ep:  132.60000000000008 53.699999999999996
Ek, Ep:  131.07000000000008 55.199999999999996
Ek, Ep:  129.72000000000006 56.7
Ek, Ep:  128.55000000000007 57.599999999999994
Ek, Ep:  127.560000000000

Ek, Ep:  25.18499999999998 153.3
Ek, Ep:  23.32499999999998 154.8
Ek, Ep:  21.644999999999985 156.3
Ek, Ep:  20.14499999999999 157.8
Ek, Ep:  18.82499999999999 159.0
Ek, Ep:  17.68499999999999 159.89999999999998
Ek, Ep:  16.724999999999994 160.8
Ek, Ep:  15.944999999999995 161.39999999999998
Ek, Ep:  15.344999999999997 161.7
Ek, Ep:  14.924999999999997 162.0
Ek, Ep:  14.684999999999997 162.3
Ek, Ep:  14.624999999999998 162.29999999999998
Ek, Ep:  14.745 162.0
Ek, Ep:  15.044999999999998 161.7
Ek, Ep:  15.524999999999999 161.1
Ek, Ep:  16.185 160.2
Ek, Ep:  17.025 159.3
Ek, Ep:  18.044999999999995 158.1
Ek, Ep:  19.24499999999999 156.9
Ek, Ep:  20.624999999999993 155.4
Ek, Ep:  22.18499999999999 153.9
Ek, Ep:  23.92499999999999 152.4
Ek, Ep:  25.84499999999999 150.6
Ek, Ep:  27.94499999999999 148.8
Ek, Ep:  30.224999999999987 146.7
Ek, Ep:  32.68499999999999 144.29999999999998
Ek, Ep:  35.32499999999998 141.89999999999998
Ek, Ep:  38.14499999999998 139.2
Ek, Ep:  41.14499999999998 136.5